In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import piemap

In [ ]:
dep = gpd.read_file('dep2022_simplify.json')

In [ ]:
reg = dep.dissolve('reg',as_index=False)

In [ ]:
sciage = pd.read_csv('sciage_afc.csv', sep=',',dtype={'REG':str, 'DEP':str, 'SIREN': str, 'DEPETAB':str,'NOM_DOSSIER':str, 'ESSENCE':str})
sciage['entreprise'] = 1

In [ ]:
typoc = {1:'tab:blue',2:'tab:orange',3:'tab:green'}
sciage['clusterc'] = sciage.cluster.apply(lambda x:typoc[x])
sciage.plot.scatter('axe1','axe2',c='clusterc', s=5, figsize=(10,10))

In [ ]:
typo = {1:'1-Feuillus', 2 : '2-Pin maritime', 3 : '3-Conifère - hors pin maritime'}
sciage['classe'] = sciage.cluster.apply(lambda x: typo[x])

In [ ]:
temp = pd.cut(x=sciage["Total"], bins=[0,5000,10000,20000,float('inf')], labels=["< 5000 m3","5000 - 10000m3", "10000 - 20000 m3", ">= 20000 m3"])
sciage =  pd.concat([sciage,pd.get_dummies(temp)],axis=1)
sciage['taille'] = temp

In [ ]:
sciage[['NOM_DOSSIER','classe']].to_csv('typologie_sciage.csv', index=False)

### Caractérisation des classes d'entreprises

#### Production de sciage réalisée par classe

In [ ]:
sciage_cluster = sciage.groupby('classe',as_index=False).sum()

In [ ]:
sciage_cluster.plot.bar(x='classe', y='Total', rot=0, figsize=(15,5), legend=False, title='Production de sciage en m3');

In [ ]:
sciage_cluster.loc['total']= sciage_cluster.sum()
sciage_cluster['production_moyenne'] = sciage_cluster.Total / sciage_cluster.entreprise
sciage_cluster[['classe',  'entreprise','Total','production_moyenne']].rename(columns={'Total':'Production totale', 'classe':'Classe', 'production_moyenne': 'Production Moyenne'})

#### Répartition de la production et des entreprises par taille

In [ ]:
temp = sciage.groupby(['classe', 'taille'], as_index=False)['entreprise','Total'].sum()
temp2 = sciage.groupby(['classe'], as_index=False)['entreprise','Total'].sum()
sciage_taille_classe = pd.merge(temp,temp2, on='classe',suffixes=('','_sum'))
sciage_taille_classe.entreprise=sciage_taille_classe.entreprise/sciage_taille_classe.entreprise_sum * 100
sciage_taille_classe.Total=sciage_taille_classe.Total/sciage_taille_classe.Total_sum * 100

In [ ]:
g = sns.catplot(
    x="taille",       # x variable name
    y="entreprise",       # y variable name
    hue="classe",  # group variable name
    data=sciage_taille_classe,     # dataframe to plot
    kind="bar",
    height=5, # make the plot 5 units high
    aspect=3,legend_out=False,
    estimator=np.sum
    
);
g.set_axis_labels("", "en %")
plt.title('Repartition des entreprises selon leur taille');

In [ ]:
g= sns.catplot(
    x="taille",       # x variable name
    y="Total",       # y variable name
    hue="classe",  # group variable name
    data=sciage_taille_classe,     # dataframe to plot
    kind="bar",
    height=5, # make the plot 5 units high
    aspect=3,legend_out=False,
    estimator=np.sum
    
);
g.set_axis_labels("", "en %")
plt.title('Repartition de la production selon leur taille');

#### Répartition de la production selon les essences par classe

In [ ]:
sciage_cluster_essence = sciage_cluster.copy()
sciage_cluster_essence[['Chêne', 'Hêtre', 'Châtaigner', 'Feuillus précieux',
       'Peuplier', 'Autres feuillus', 'Sapin-épicéa', 'Douglas', 'Mélèze',
       'Pin maritime', 'Autres pins', 'Autres conifères', 'Bois tropicaux']]=sciage_cluster_essence[['Chêne', 'Hêtre', 'Châtaigner', 'Feuillus précieux',
       'Peuplier', 'Autres feuillus', 'Sapin-épicéa', 'Douglas', 'Mélèze',
       'Pin maritime', 'Autres pins', 'Autres conifères', 'Bois tropicaux']].div(sciage_cluster_essence.Total,axis=0) * 100

sciage_cluster_essence['Autres feuillus'] =  sciage_cluster_essence[['Châtaigner', 'Feuillus précieux', 'Autres feuillus']].sum(axis=1)
sciage_cluster_essence['Autres conifères'] =  sciage_cluster_essence[['Mélèze', 'Autres pins', 'Autres conifères']].sum(axis=1)

sciage_cluster_essence=sciage_cluster_essence[['classe','Chêne', 'Hêtre', 'Peuplier', 'Autres feuillus', 'Sapin-épicéa', 'Douglas', 'Pin maritime', 'Autres conifères']]

In [ ]:
temp = []
for x in sciage_cluster_essence.iloc[:3].groupby('classe',as_index=False):
    df = x[1].transpose()
    df['classe'] = x[0]
    df['essence'] = df.index
    df = df.drop('classe')
    df.columns = ['Total','classe','essence']
    temp.append(df)
sciage_cluster_essence = pd.concat(temp)

In [ ]:
sciage_cluster_essence.index=sciage_cluster_essence.classe

In [ ]:
g= sns.catplot(
    x="essence",       # x variable name
    y="Total",       # y variable name
    hue="classe",  # group variable name
    data=sciage_cluster_essence,     # dataframe to plot
    kind="bar",
    height=5, # make the plot 5 units high
    aspect=3,legend_out=False
    
);
g.set_axis_labels("", "en %")
plt.title('Profil des classes selon les essences sciées');

#### Part de la production départementale réalisée par classe d'entreprises

In [ ]:
sciage = pd.concat([sciage,pd.get_dummies(sciage.classe).mul(sciage.Total, axis=0)], axis=1)

In [ ]:
sciage_cluster_dep = sciage.groupby(['DEP'],as_index=False)['classe 1 : Feuillus', 'classe 2 : Pin maritime',
       'classe 3 : Conifère - hors pin maritime', 'Total'].sum()

In [ ]:
sciage_cluster_reg = sciage_cluster_dep.merge(dep[['dep','reg']], left_on='DEP', right_on='dep')
sciage_cluster_reg = sciage_cluster_reg.groupby('reg',as_index=False)['classe 1 : Feuillus', 'classe 2 : Pin maritime',
       'classe 3 : Conifère - hors pin maritime'].sum()

In [ ]:
sciage_cluster_reg = reg.merge(sciage_cluster_reg, left_on='reg', right_on='reg')

In [ ]:
from importlib import reload
piemap = reload(piemap)

In [ ]:
temp = piemap.piemap(sciage_cluster_reg, ['classe 1 : Feuillus', 'classe 2 : Pin maritime',
       'classe 3 : Conifère - hors pin maritime'])
typocreg = {0:'tab:blue',1:'tab:orange',2:'tab:green'}
temp['color'] = temp.quantity.apply(lambda x: typocreg[x])

In [ ]:
import matplotlib.patches as patches
ax = reg[~reg.reg.isin(['01','02','03','04','06','94'])].plot(figsize=(20,20), color='lightgrey', zorder=1)
reg.boundary.plot(color='white',ax=ax,zorder=2)
temp.plot(color=temp.color,ax=ax, legend=True, categorical=True,zorder=3, alpha=0.8)
ax.set_axis_off()
ax.set_title('Répartition de la production de sciage')
list_of_lito = ['Feuillus', 'Pin maritime', 'Conifère - hors pin maritime']

color_dict = {'Feuillus':'tab:blue','Pin maritime':'tab:orange','Conifère - hors pin maritime':'tab:green'}

# Creates a rectangular patch for each contaminant, using the colors above
patch_list =[]
for lito in list_of_lito:
    label = lito.capitalize()
    color = color_dict[lito]
    patch_list.append(patches.Patch(facecolor=color, 
                                    label=label, 
                                    alpha=0.9, 
                                    linewidth=1, 
                                    edgecolor=None))

# Creates a legend with the list of patches above.
ax.legend(handles=patch_list, fontsize=14, loc='upper right',
        bbox_to_anchor = (0.3,0.95),title='Classe', title_fontsize=14)

In [ ]:
sciage_cluster_dep['classe 1 : Feuillus'] = sciage_cluster_dep['classe 1 : Feuillus'] / sciage_cluster_dep.Total * 100
sciage_cluster_dep['classe 2 : Pin maritime'] = sciage_cluster_dep['classe 2 : Pin maritime'] / sciage_cluster_dep.Total * 100
sciage_cluster_dep['classe 3 : Conifère - hors pin maritime'] = sciage_cluster_dep['classe 3 : Conifère - hors pin maritime'] / sciage_cluster_dep.Total * 100
sciage_cluster_dep = dep.merge(sciage_cluster_dep, left_on='dep', right_on='DEP')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,7))
fig.suptitle("Part de la production départementale réalisée par classe d'entreprises")
sciage_cluster_dep.plot('classe 1 : Feuillus',scheme='userdefined', cmap='Blues', legend=True, legend_kwds=dict(fmt='{:.0f}', interval=True), figsize=(10,10), classification_kwds={'bins':[25,50,75]}, ax=ax1)
dep[~dep.dep.isin(['2A','2B', '971', '972', '973', '974', '975', '976'])].boundary.plot(ax=ax1, color='grey')
ax1.set_axis_off()
ax1.set_title('classe 1 : Feuillus')
sciage_cluster_dep.plot('classe 2 : Pin maritime',scheme='userdefined', cmap='Oranges', legend=True, legend_kwds=dict(fmt='{:.0f}', interval=True), figsize=(10,10), classification_kwds={'bins':[25,50,75]}, ax=ax2)
dep[~dep.dep.isin(['2A','2B', '971', '972', '973', '974', '975', '976'])].boundary.plot(ax=ax2, color='grey')
ax2.set_axis_off()
ax2.set_title('classe 2 : Pin maritime')
sciage_cluster_dep.plot('classe 3 : Conifère - hors pin maritime',scheme='userdefined', cmap='Greens', legend=True, legend_kwds=dict(fmt='{:.0f}', interval=True), figsize=(10,10), classification_kwds={'bins':[25,50,75]}, ax=ax3)
dep[~dep.dep.isin(['2A','2B', '971', '972', '973', '974', '975', '976'])].boundary.plot(ax=ax3, color='grey')
ax3.set_axis_off()
ax3.set_title('classe 3 : Conifère - hors pin maritime');

### L'approvisionnement des scieries

In [ ]:
approv = pd.read_csv('APPROV_STOCKS.csv', sep=';', dtype={'NOM_DOSSIER':str})
approv = approv.fillna(0)

In [ ]:
approv.columns

In [ ]:
nom_approv = {1:'Exploitant scieur', 0:'Scieur exclusif'}
approv['exploitant_scieur_b'] = (approv.APENT_FE + approv.APENT_CO +approv.APENT_TROP)>1
approv['type'] = approv['exploitant_scieur_b'].apply(lambda x:nom_approv[x])

In [ ]:
sciage_approv = sciage.merge(approv[['NOM_DOSSIER', 'type','exploitant_scieur_b']], on='NOM_DOSSIER', how='left')
print(sciage_approv['exploitant_scieur_b'].sum()/sciage_approv.entreprise.sum() * 100)

In [ ]:
sciage_approv = pd.concat([sciage_approv,pd.get_dummies(sciage_approv.type)], axis=1)
sciage_approv['Exploitant scieur production'] = sciage_approv['Exploitant scieur'] * sciage_approv.Total
sciage_approv['Scieur exclusif production'] = sciage_approv['Scieur exclusif'] * sciage_approv.Total

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe'],as_index=False)[['Exploitant scieur','Scieur exclusif', 'entreprise',
                                                                         'Exploitant scieur production','Scieur exclusif production','Total']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
sciage_approv_classe['Exploitant scieur'] = sciage_approv_classe['Exploitant scieur'] / sciage_approv_classe.entreprise * 100
sciage_approv_classe['Scieur exclusif'] = sciage_approv_classe['Scieur exclusif'] / sciage_approv_classe.entreprise * 100
sciage_approv_classe['Exploitant scieur production'] = sciage_approv_classe['Exploitant scieur production'] / sciage_approv_classe.Total * 100
sciage_approv_classe['Scieur exclusif production'] = sciage_approv_classe['Scieur exclusif production'] / sciage_approv_classe.Total * 100

In [ ]:
sciage_approv_classe[['classe','Exploitant scieur','Scieur exclusif', 'Exploitant scieur production','Scieur exclusif production',]]

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe', 'taille'], as_index=False)['exploitant_scieur_b','entreprise'].sum()
sciage_approv_classe.exploitant_scieur_b=sciage_approv_classe.exploitant_scieur_b/sciage_approv_classe.entreprise * 100

In [ ]:
g= sns.catplot(
    x="taille",       # x variable name
    y="exploitant_scieur_b",       # y variable name
    hue="classe",  # group variable name
    data=sciage_approv_classe,     # dataframe to plot
    kind="bar",
    height=5, # make the plot 5 units high
    aspect=3,legend_out=False
    
);
g.set_axis_labels("", "en %")
plt.title('Part des exploitants scieurs par classe et taille');

### Origine approvisionnement en grumes sur écorce

In [ ]:
inertie = pd.read_csv('inertie.csv', dtype={'NOM_DOSSIER':str}).fillna(0)

In [ ]:
sciage_approv = sciage.merge(approv, on='NOM_DOSSIER', how='left')
sciage_approv = sciage_approv.merge(inertie, on='NOM_DOSSIER', how='left')

In [ ]:
sciage_approv_inertie = sciage_approv.groupby('classe',as_index=False)['max','sum'].mean()

In [ ]:
sciage_approv_inertie['mdep'] = sciage_approv_inertie['max'] / sciage_approv_inertie['sum'] * 100

In [ ]:
sciage_approv_inertie

In [ ]:
sciage_approv['APENT'] = sciage_approv.APENT_FE + sciage_approv.APENT_CO + sciage_approv.APENT_TROP
sciage_approv['APACHSCI'] = sciage_approv['APACHSCI_FE'] + sciage_approv['APACHSCI_CO'] + sciage_approv['APACHSCI_TROP']
sciage_approv['APACHEXF'] = sciage_approv['APACHEXF_FE'] + sciage_approv['APACHEXF_CO'] + sciage_approv['APACHEXF_TROP']
sciage_approv['DONT_APPRO_121'] = sciage_approv[['DONT_APPRO_FE_121', 'DONT_APPRO_CO_121']].sum(axis=1)
sciage_approv['DONT_APPRO_122'] = sciage_approv[['DONT_APPRO_FE_122', 'DONT_APPRO_CO_122']].sum(axis=1)
sciage_approv.DTDEP_FE = sciage_approv.DTDEP_FE * sciage_approv.APACHEXF_FE / 100
sciage_approv.DTDEP_CO = sciage_approv.DTDEP_CO * sciage_approv.APACHEXF_CO / 100
sciage_approv['DTDEP']= sciage_approv[['DTDEP_FE', 'DTDEP_CO']].sum(axis=1)

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe'], as_index=False)[['APENT','APACHSCI','APACHEXF','DONT_APPRO_121','DONT_APPRO_122','DTDEP']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
total = (sciage_approv_classe.APENT + sciage_approv_classe.APACHSCI + sciage_approv_classe.APACHEXF)
sciage_approv_classe['DONT_APPRO_121'] = sciage_approv_classe.DONT_APPRO_121 / sciage_approv_classe.APACHSCI * 100
sciage_approv_classe['DONT_APPRO_122'] = sciage_approv_classe.DONT_APPRO_122 / sciage_approv_classe.APACHEXF * 100
sciage_approv_classe['DTDEP'] = sciage_approv_classe.DTDEP / sciage_approv_classe.APACHEXF * 100
sciage_approv_classe.APENT = sciage_approv_classe.APENT / total  * 100
sciage_approv_classe.APACHSCI = sciage_approv_classe.APACHSCI / total * 100
sciage_approv_classe.APACHEXF = sciage_approv_classe.APACHEXF / total * 100
sciage_approv_classe.at['total','classe'] = 'Total'
sciage_approv_classe

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe','type'], as_index=False)[['APENT','APACHSCI','APACHEXF','DONT_APPRO_121','DONT_APPRO_122','DTDEP']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
total = (sciage_approv_classe.APENT + sciage_approv_classe.APACHSCI + sciage_approv_classe.APACHEXF)
sciage_approv_classe['DONT_APPRO_121'] = sciage_approv_classe.DONT_APPRO_121 / sciage_approv_classe.APACHSCI * 100
sciage_approv_classe['DONT_APPRO_122'] = sciage_approv_classe.DONT_APPRO_122 / sciage_approv_classe.APACHEXF * 100
sciage_approv_classe['DTDEP'] = sciage_approv_classe.DTDEP / sciage_approv_classe.APACHEXF * 100
sciage_approv_classe.APENT = sciage_approv_classe.APENT / total  * 100
sciage_approv_classe.APACHSCI = sciage_approv_classe.APACHSCI / total * 100
sciage_approv_classe.APACHEXF = sciage_approv_classe.APACHEXF / total * 100
sciage_approv_classe.at['total','classe'] = 'Total'
sciage_approv_classe

In [ ]:
sciage_approv['CO'] = sciage_approv[['APENT_CO','APACHSCI_CO','APACHEXF_CO']].sum(axis=1)
sciage_approv['FE'] = sciage_approv[['APENT_FE','APACHSCI_FE','APACHEXF_FE']].sum(axis=1)

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe'], as_index=False)[['CO','FE']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
total = (sciage_approv_classe.CO + sciage_approv_classe.FE)
sciage_approv_classe.CO = sciage_approv_classe.CO / total  * 100
sciage_approv_classe.FE = sciage_approv_classe.FE / total * 100
sciage_approv_classe.at['total','classe'] = 'Total'
sciage_approv_classe

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe','taille'], as_index=False)[['CO','FE']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
total = (sciage_approv_classe.CO + sciage_approv_classe.FE)
sciage_approv_classe.CO = sciage_approv_classe.CO / total  * 100
sciage_approv_classe.FE = sciage_approv_classe.FE / total * 100
sciage_approv_classe.at['total','classe'] = 'Total'
sciage_approv_classe

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe','type'], as_index=False)[['CO','FE']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
total = (sciage_approv_classe.CO + sciage_approv_classe.FE)
sciage_approv_classe.CO = sciage_approv_classe.CO / total  * 100
sciage_approv_classe.FE = sciage_approv_classe.FE / total * 100
sciage_approv_classe.at['total','classe'] = 'Total'
sciage_approv_classe

In [ ]:
sciage_approv_classe = sciage_approv.groupby(['classe','type','taille'], as_index=False)[['CO','FE']].sum()
sciage_approv_classe.loc['total']= sciage_approv_classe.sum()
total = (sciage_approv_classe.CO + sciage_approv_classe.FE)
sciage_approv_classe.CO = sciage_approv_classe.CO / total  * 100
sciage_approv_classe.FE = sciage_approv_classe.FE / total * 100
sciage_approv_classe.at['total','classe'] = 'Total'
sciage_approv_classe

In [ ]:
115+48+24+42+38+28+5+4

In [ ]:
sciage_approv_taille = sciage_approv.groupby(['taille'], as_index=False)[['APENT','APACHSCI','APACHEXF']].sum()
total = (sciage_approv_taille.APENT + sciage_approv_taille.APACHSCI + sciage_approv_taille.APACHEXF)
sciage_approv_taille.APENT = sciage_approv_taille.APENT / total  * 100
sciage_approv_taille.APACHSCI = sciage_approv_taille.APACHSCI / total * 100
sciage_approv_taille.APACHEXF = sciage_approv_taille.APACHEXF / total * 100

In [ ]:
sciage_approv_taille

In [ ]:
sciage_approv_classe_taille = sciage_approv.groupby(['classe','taille',], as_index=False)[['APENT','APACHSCI','APACHEXF','DONT_APPRO_121','DONT_APPRO_122','DTDEP']].sum()
sciage_approv_classe_taille['DONT_APPRO_121'] = sciage_approv_classe_taille.DONT_APPRO_121 / sciage_approv_classe_taille.APACHSCI * 100
sciage_approv_classe_taille['DONT_APPRO_122'] = sciage_approv_classe_taille.DONT_APPRO_122 / sciage_approv_classe_taille.APACHEXF * 100
sciage_approv_classe_taille['DTDEP'] = sciage_approv_classe_taille.DTDEP / sciage_approv_classe_taille.APACHEXF * 100
total = (sciage_approv_classe_taille.APENT + sciage_approv_classe_taille.APACHSCI + sciage_approv_classe_taille.APACHEXF)
sciage_approv_classe_taille.APENT = sciage_approv_classe_taille.APENT / total  * 100
sciage_approv_classe_taille.APACHSCI = sciage_approv_classe_taille.APACHSCI / total * 100
sciage_approv_classe_taille.APACHEXF = sciage_approv_classe_taille.APACHEXF / total * 100

In [ ]:
sciage_approv_classe_taille